[View in Colaboratory](https://colab.research.google.com/github/Romanm87/TGS_Salt/blob/master/conv3.ipynb)

In [1]:
import numpy as np
import pandas as pd
from keras.layers import Input, Conv2D
from keras.models import Model
from keras import optimizers
from keras import backend as K
from sklearn.metrics import confusion_matrix
from google.colab import files
import tensorflow as tf
import cv2
import os
import json
import zipfile

Using TensorFlow backend.


In [2]:
# Use kaggle API

!pip install kaggle
api_token = {"username":"romanm87","key":"###################"}

os.chdir('/')
!mkdir ~/.kaggle #kaggle API searches in root directory for .kaggle/kaggle.json

with open('/root/.kaggle/kaggle.json', 'w') as file:
    json.dump(api_token, file)
!chmod 600 /root/.kaggle/kaggle.json

# API link from Kaggle:
!kaggle competitions download -c tgs-salt-identification-challenge

zip_ref = zipfile.ZipFile('train.zip', 'r')
zip_ref.extractall()
zip_ref.close()


    100% |████████████████████████████████| 61kB 3.9MB/s 
    100% |████████████████████████████████| 235kB 7.1MB/s 
  Running setup.py bdist_wheel for kaggle ... - \ done
  Stored in directory: /root/.cache/pip/wheels/44/2c/df/22a6eeb780c36c28190faef6252b739fdc47145fd87a6642d4
  Running setup.py bdist_wheel for python-slugify ... - done
  Stored in directory: /root/.cache/pip/wheels/e3/65/da/2045deea3098ed7471eca0e2460cfbd3fdfe8c1d6fa6fcac92
Successfully built kaggle python-slugify
  0% 0.00/322k [00:00<?, ?B/s]
100% 322k/322k [00:00<00:00, 72.0MB/s]
  0% 0.00/264k [00:00<?, ?B/s]
100% 264k/264k [00:00<00:00, 71.4MB/s]
  0% 0.00/922k [00:00<?, ?B/s]
100% 922k/922k [00:00<00:00, 83.1MB/s]
 91% 148M/163M [00:01<00:00, 93.0MB/s]
100% 163M/163M [00:01<00:00, 98.3MB/s]
 53% 20.0M/37.9M [00:00<00:00, 40.4MB/s]
100% 37.9M/37.9M [00:00<00:00, 112MB/s] 


In [0]:
##### Prepare data and model
tr_image_dir = os.chdir('/images')
train_im = os.listdir(tr_image_dir)
x = np.array([np.array(cv2.imread(p, cv2.IMREAD_GRAYSCALE)) for p in train_im]) / 255

tr_masks_dir = os.chdir('/masks')
train_ma = os.listdir(tr_masks_dir)
y = np.array([np.array(cv2.imread(p, cv2.IMREAD_GRAYSCALE)) for p in train_ma]) / 255

# expand dimensions for CNN inout
x = np.expand_dims(x, axis=3)
y = np.expand_dims(y, axis=3)

# split training vs validation set
train_val_split = 0.1
x_train = x[0:int(x.shape[0]*(1-train_val_split)),:,:,:]
y_train = y[0:int(y.shape[0]*(1-train_val_split)),:,:,:]
x_val = x[int(x.shape[0]*(1-train_val_split)):,:,:,:]
y_val = y[int(y.shape[0]*(1-train_val_split)):,:,:,:]

def conv_block(x_input, num_layers, f, k):
    x = x_input
    for l in range(num_layers):
        x = Conv2D(f, (k, k), padding = 'SAME', activation = 'relu')(x)
    return x

def conv_net(input_shape, num_layers, num_filters, kernel_sizes):
    x_input = Input(input_shape)
    x = conv_block(x_input, num_layers[0], num_filters[0], kernel_sizes[0])
    x = conv_block(x, num_layers[1], num_filters[1], kernel_sizes[1])
    x = conv_block(x, num_layers[2], num_filters[2], kernel_sizes[2])
    x = conv_block(x, num_layers[3], num_filters[3], kernel_sizes[3])
    x = conv_block(x, num_layers[4], num_filters[4], kernel_sizes[4])
    x = Conv2D(1, (1,1), activation = 'sigmoid')(x)
    model = Model(inputs = x_input, outputs=x)
    return model

#input_shape = Input(shape = (101,101,1))
input_shape = (101, 101, 1)
num_layers = [1, 1, 1, 1, 1] #[5, 5, 5, 5, 5]
num_filters = [16, 8, 8, 4, 2] #[32, 24, 16, 8, 4]
kernel_sizes = [3, 5, 7, 9, 11]

In [0]:
##### Define 

class MeanIoU(object):
    def __init__(self):
        super().__init__()
    def mean_iou(self, y_true, y_pred):
        # Wraps np_mean_iou method and uses it as a TensorFlow op.
        # Takes numpy arrays as its arguments and returns numpy arrays as outputs
        return tf.py_func(self.np_mean_iou, [y_true, y_pred], tf.float64)
    def np_mean_iou(self, y_true, y_pred):
        y_pred = np.round(y_pred + 0.05, 0).reshape(-1)
        y_true = y_true.reshape(-1)        
        conf = confusion_matrix(y_pred, y_true)        
        # Compute the IoU and mean IoU from the confusion matrix:
        true_positive = conf[1,1]
        false_positive = conf[1,0]
        false_negative = conf[0,1]
        # Just in case we get a division by 0, ignore/hide the error and set the value to 0
        with np.errstate(divide='ignore', invalid='ignore'):
            iou = true_positive / (true_positive + false_positive + false_negative)
        #iou[np.isnan(iou)] = 0
        return np.mean(iou).astype(np.float64)
miou = MeanIoU()

def bin_acc05(y_true, y_pred):
    return K.mean(K.equal(y_true, K.round(y_pred + 0.05)), axis=-1)

In [5]:
##### Define model 
model = conv_net(input_shape, num_layers, num_filters, kernel_sizes)
adam = optimizers.Adam(lr = 0.001)
model.compile(loss = 'binary_crossentropy', optimizer='adam', 
              metrics=[bin_acc05, miou.mean_iou])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 101, 101, 1)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 101, 101, 16)      160       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 101, 101, 8)       3208      
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 101, 101, 8)       3144      
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 101, 101, 4)       2596      
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 101, 101, 2)       970       
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 101, 101, 1)       3         
Total para

In [6]:
##### Run model
model.fit(x_train, y_train, epochs=5, batch_size=128, validation_data=(x_val,y_val), verbose=1)

Train on 3600 samples, validate on 400 samples
Epoch 1/5
3600/3600 [==============================] - 39s 11ms/step - loss: 0.6901 - bin_acc05: 0.2453 - mean_iou: 0.2453 - val_loss: 0.6869 - val_bin_acc05: 0.2717 - val_mean_iou: 0.2717
Epoch 2/5
3600/3600 [==============================] - 35s 10ms/step - loss: 0.6829 - bin_acc05: 0.2453 - mean_iou: 0.2453 - val_loss: 0.6807 - val_bin_acc05: 0.2717 - val_mean_iou: 0.2717
Epoch 3/5
3600/3600 [==============================] - 35s 10ms/step - loss: 0.6761 - bin_acc05: 0.2453 - mean_iou: 0.2453 - val_loss: 0.6747 - val_bin_acc05: 0.2717 - val_mean_iou: 0.2717
Epoch 4/5
3600/3600 [==============================] - 34s 10ms/step - loss: 0.6695 - bin_acc05: 0.2453 - mean_iou: 0.2453 - val_loss: 0.6692 - val_bin_acc05: 0.2717 - val_mean_iou: 0.2717
Epoch 5/5
3600/3600 [==============================] - 34s 10ms/step - loss: 0.6632 - bin_acc05: 0.2453 - mean_iou: 0.2453 - val_loss: 0.6637 - val_bin_acc05: 0.2717 - val_mean_iou: 0.2717


In [0]:
##### Predict results
y_train_pred = model.predict(x_train, verbose=1)
y_val_pred = model.predict(x_val, verbose=1)

# evaluate on local computer
#true_positive_train = []
#false_positive_train = []
#false_negative_train = []
#true_positive_val = []
#false_positive_val = []
#false_negative_val = []
#thresh = [0.025, 0.05, 0.075, 0.1, 0.125, 0.15, 0.175, 0.2, 0.225, 0.25, 0.275, 0.3]
#for i in thresh:
#    y_train_pred_bin = np.round(y_train_pred + i, 0)
#    y_val_pred_bin = np.round(y_val_pred + i, 0)
#    conf_train = confusion_matrix(list(y_train_pred_bin.reshape(-1)), list(y_train.reshape(-1)))
#    true_positive_train.append(conf_train[1,1])
#    false_positive_train.append(conf_train[1,0])
#    false_negative_train.append(conf_train[0,1])
#    iou_train.append(conf_train[1,1]/(conf_train[1,1]+conf_train[1,0]+conf_train[0,1]))
#    conf_val = confusion_matrix(list(y_val_pred_bin.reshape(-1)), list(y_val.reshape(-1)))
#    true_positive_val.append(conf_val[1,1])
#    false_positive_val.append(conf_val[1,0])
#    false_negative_val.append(conf_val[0,1])
#    iou_val.append(conf_val[1,1]/(conf_val[1,1]+conf_val[1,0]+conf_val[0,1]))

#print('true pos train', true_positive_train)
#print('false pos train', false_positive_train)
#print('false neg train', false_negative_train)
#print('true pos val', true_positive_val)
#print('false pos val', false_positive_val)
#print('false neg val', false_negative_val)

In [0]:
###### Download results to local computer for evaluation
from google.colab import files
results_train = pd.DataFrame({'y_train': y_train.reshape(-1),'y_train_pred': y_train_pred.reshape(-1)})
results_train.to_csv("results_train.csv")
results_val = pd.DataFrame({'y_val': y_val.reshape(-1),'y_val_pred': y_val_pred.reshape(-1)})
results_val.to_csv("results_val.csv")
files.download('results_train.csv')
files.download('results_val.csv')